# Lisbon Airbnb Data

## An exploratory analysis

Here will be analyzed the data provided regarding Airbnb bookings and property listings. In this analysis the following questions will be addressed:
- Who is booking Airbnb rooms in Portugal? (Profiling)
- Where do they choose to stay? (Geographic analysis)
- When do they do it? (Time Series analysis)

All the data was provided by Nova School of Business and Economics' research group: **Data Science for Social Impact and Management**.

This analysis is focused on Tourism Management in Portugal and was developed for Turismo de Portugal, a public national institution responsible for promote, enrich and sustain Tourism in the country.

The report is structured as follows:


- **[Preprocessing and Data Exploration](#Preprocessing-and-Data-Exploration)**
    - [Property listings](#Property-listings)
    - [Daily Bookings](#Daily-Bookings)
    - [Monthly Bookings](#Monthly-Bookings)
    - [Listing Reviews](#Listing-Reviews)
    - [Outlier Detection](#Outlier-Detection)
    - [Missing Values Detection](#Missing-Values-Detection)
    - [Correlation Matrices](#Correlation-Matrices)

- **[Modelling](#Modelling)**
    - [Profiling](#Profiling)

- **[Analyst reviews output/Evaluation](#Analyst-reviews-output/Evaluation)**



## Preprocessing and Data Exploration

Let's start by importing some necessary libraries and specify tokens and configurations:

In [ ]:
import pandas as pd
import numpy as np
import datetime
from numba import jit
import sklearn.cluster as sk_cluster
from sklearn import preprocessing, neighbors
import fiona
from shapely.geometry import Point, shape

# visualizations
import plotly.graph_objs as go
import plotly
import plotly.tools as tls # testing function to see wether it allows html to be embedded
import matplotlib
import matplotlib.pyplot as plt

plotly.offline.init_notebook_mode()
pd.options.mode.chained_assignment = None # default = 'warn'
mapbox_token = 'pk.eyJ1Ijoiam9hb2ZvbnNlY2EiLCJhIjoiY2picXB3cDVvMDczYjJ3bzBxNDV3dGI0MSJ9.XpQDNjTuMAM-xckGln0KrA'

preprocess_dir = '_lisbon_data_out/'
viz_out = '_lisbon_viz_out/'
plt_viz = '_lisbon_plt_viz/'
shp_dir = 'districts_shp/'

In [ ]:
properties = pd.read_csv('../data/Portugal_Property_2018-02-02.csv')
daily = pd.read_csv('../data/Portugal_Daily_2018-02-02.csv')
monthly = pd.read_csv('../data/Portugal_Monthly_2018-02-02.csv')
reviews = pd.read_csv('../data/Portugal_Review_2018-02-02.csv')

## Pre select area of analysis

In [ ]:

print('Opening shapefile with districts\' borders...')
shp_file = fiona.open(shp_dir+'PRT_adm1.shp')

distritos = {0:'Evora', 1:'Aveiro', 2: 'Açores', 3: 'Beja', 4: 'Braga', 5: 'Bragança', 6: 'Castelo Branco',
             7: 'Coimbra', 8: 'Faro', 9: 'Guarda', 10: 'Leiria', 11: 'Lisboa', 12: 'Madeira', 13: 'Portalegre',
            14: 'Porto', 15: 'Santarem', 16: 'Setubal', 17: 'Viana do Castelo', 18: 'Vila Real', 19: 'Viseu',
            30:'unknown' 
            }

print('Formatting shapefile and defining polygons and points...')
shp_final = []
for distrito in shp_file:
    distrito.update({'shape':shape(distrito['geometry'])})
    shp_final.append(distrito)

def generate_point(row):
    return Point(row['Longitude'],row['Latitude'])

properties['point'] = properties.apply(generate_point, axis=1)


def check_district(row):
    for distrito in shp_final:
        check = distrito['shape'].contains(row['point'])
        if check:
            return distrito['id']

print('Assigning the respective district id\'s to the points...')
properties['cluster'] = properties.apply(check_district, axis=1)

print('Assigning district id\'s to points outside of borders (i.e., the ones with imprecisions in coordinates)...')
properties['cluster'] = properties['cluster'].fillna(30).astype('int')
n_neighbors = 3
X = properties[properties['cluster'] != 30][['Latitude', 'Longitude']]
y = properties[properties['cluster'] != 30]['cluster']
clf = neighbors.KNeighborsClassifier(n_neighbors)
clf.fit(X, y)
knn_prediction = clf.predict(properties[properties['cluster'] == 30][['Latitude', 'Longitude']])
properties.loc[properties['cluster'] == 30, 'cluster'] = knn_prediction

print('Assigning district names to id\'s')
def get_area_name(value):
    return distritos[value]

properties['cluster_2'] = properties['cluster'] 
properties['cluster'] = properties['cluster'].apply(get_area_name)




In [ ]:

print('Filtering out values outside of Porto...')
properties_porto = properties[properties['cluster']=='Porto']
property_ids_porto = list(properties_porto['Property ID'])
reviews_porto = reviews.loc[reviews['Property ID'].isin(property_ids_porto)]
daily_porto = daily.loc[daily['Property ID'].isin(property_ids_porto)]
monthly_porto = monthly.loc[monthly['Property ID'].isin(property_ids_porto)]

print('Filtering out values outside of Lisbon...')
properties_lisbon = properties[properties['cluster']=='Lisboa']
property_ids_lisbon = list(properties_lisbon['Property ID'])
reviews_lisbon = reviews[reviews['Property ID'].isin(property_ids_lisbon)]
daily_lisbon = daily[daily['Property ID'].isin(property_ids_lisbon)]
monthly_lisbon = monthly[monthly['Property ID'].isin(property_ids_lisbon)]

print('Done!')


In [ ]:
tables = dict(properties = properties,
              daily      = daily,
              monthly    = monthly,
              reviews    = reviews)

for table in ['properties', 'daily', 'monthly', 'reviews']:
    print(('Columns in table %s:\n' % table ), list(tables[table]))
    print('Total number of variables in dataset: ', len(list(tables[table])),'\n')

___
___
# Lisbon
___
___

In [ ]:
properties = properties_lisbon
reviews = reviews_lisbon
daily = daily_lisbon
monthly = monthly_lisbon

### Property listings

We'll start by analysing the **Properties table**.
It contains thorough information regarding Property listings in the Airbnb platform, from which we will use only a subset of this data:

In [ ]:
props_df = properties[['Property ID', 'Property Type', 'Listing Type', 'Bedrooms', 'Created Date', 'City',
                       'Annual Revenue LTM (USD)', 'Average Daily Rate (Native)', 'Average Daily Rate (USD)', 
                       'Number of Bookings LTM', 'Max Guests', 'Latitude', 'Longitude']]

Let's assess how the listings are distributed across Portugal:

In [ ]:
data = go.Data([
        go.Scattermapbox(
                lat = properties['Latitude'],
                lon = properties['Longitude'],
                mode='markers',
                marker=go.Marker(size= 2),
                text = properties['Annual Revenue LTM (USD)']
                )
        ])

layout = go.Layout(
    title='Location of listings',
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_token,
        bearing=0,
        style='light',
        center=dict(
            lat=39.64,
            lon=-7.95,
        ),
        pitch=0,
        zoom=4.2
    ),
)

wonder_map = go.Figure(data=data, layout=layout)

#generate interactive visualization
plotly.offline.plot(wonder_map, filename=viz_out+'geographic_analysis_plot.html', show_link=False, auto_open=False)

#div_file = plotly.offline.plot(wonder_map, include_plotlyjs=False, output_type='div')

#with open('divs/geographic_analysis_plot.html', 'w') as file:
#    file.write(div_file)

#show plot in notebook:
plotly.offline.iplot(wonder_map, show_link=False)

In [ ]:
# plot class variables
for column in ['Property Type', 'Listing Type']:
        # plot bar chart
        props_df[column].value_counts().plot(kind='bar', figsize=(22, 6), title=column)
        #plt.set_title(column)
        if column == 'City':
            plt.savefig(plt_viz+'listing_cities.png')
        if column == 'Listing Type':
            plt.savefig(plt_viz+'listing_types.png')
        plt.show()

In [ ]:
# some properties had zero bedrooms, which is clearly a mistake
properties['Bedrooms'] = properties['Bedrooms'].replace(to_replace=0.0, value=1.0)



### Daily Bookings

The Daily table contains bookings made in the platform in Portugal on a daily basis. As assessed previously, this is the data available in the table:

['Property ID', 'Date', 'Status', 'Booked Date', 'Price (USD)', 'Price (Native)', 'Currency Native', 'Reservation ID'] 

In [ ]:
# Select the columns we'll be using:
day_df = daily[[ 'Property ID', 'Date', 'Booked Date', 'Price (USD)', 'Reservation ID' ]]

We will now format the remaining columns to their proper formats and check for extreme values or outliers in the remaining data:

In [ ]:
day_df['Booked Date'] = pd.to_datetime(day_df['Booked Date'])
day_df['Date'] = pd.to_datetime(day_df['Date'])
# Converting Created Date to an integer format to allow plotting
day_df['booked_date_format'] = pd.DatetimeIndex(day_df['Booked Date']).astype(np.int64)
day_df['date_format'] = pd.DatetimeIndex ( day_df['Date'] ).astype ( np.int64 )

print( 'Number of observations in dataset: ', len(day_df['Reservation ID']) )

# plotting interval variables (numba.jit is used here to accelerate the processing time)
@jit
def generate_viz_daily():
    column = 'Price (USD)'
    # plot histogram
    plt.subplot(1,2,1)
    plt.hist(x=day_df[column].dropna(), bins=100)
    # plot box plot
    plt.subplot(1,2,2)
    plt.boxplot(x=day_df[column].dropna())
    # Plot configurations
    plt.subplots_adjust(wspace = 0.4)
    plt.suptitle(column)
    plt.show()
    # Computing IQR
    Q1 = day_df[column].dropna().quantile(0.25)
    Q3 = day_df[column].dropna().quantile(0.75)
    IQR = Q3 - Q1
    # Filtering Values between Q1-1.5IQR and Q3+1.5IQR
    _filtered = day_df[ Q1-1.5*IQR < day_df[column]]
    filtered = _filtered[ _filtered[column] < Q3+1.5*IQR ]
    if column == 'date_format':
        plt.savefig(plt_viz+'daily_date.png')

    if column == 'booked_date_format':
        print('Minimum value: ', min(day_df['Booked Date'].dropna()), '   Maximum Value: ', max(day_df['Booked Date'].dropna()))
    elif column == 'date_format':
        print('Minimum value: ', min(day_df['Date']), '   Maximum Value: ', max(day_df['Date']))
    else:
        print('Minimum value: ', min(day_df[column]), '   Maximum Value: ', max(day_df[column]))
    print('Count of observations outside of Inter Quartile Range: ', len(day_df[column].dropna())-len(filtered[column].dropna()))

generate_viz_daily()



(data analysis, refer to outliers, extreme values, distribution, etc)

During Clustering: Decide cutoff levels for extreme values



### Listing Reviews

The Listing Reviews table contains reviews made by users in the platform in Portugal on a monthly basis, as well as some details regarding the user's profile, namely First name, Country of origin, State (if the user is from the USA), City of origin, brief user description, last attended teaching institution and occupation. Again, this is the data available in the table, from which we will select only the variables that are relevant for this analysis:

['Property ID', 'Latitude', 'Longitude', 'Address', 'Review Date', 'Review Text', 'User ID', 'Member Since', 'First Name', 'Country', 'State', 'City', 'Description', 'School', 'Work', 'Profile Image URL', 'Profile URL']

We realize a very important detail after a first analysis: Data regarding customer profile is not standardized. In order to save space, this first analysis will not be displayed, but rather a final analysis, after preprocessing. So, we will be required to standardize it by parsing each person's country of origin from the input text received:

In [ ]:
# use script developed: reviews_data_standardized.py
# This script uses geograpy, a library that helps parsing locations from text.
# As we are using python 3.6.3 and geograpy was developed for python 2.7, we will be required to use an adapted version
# This version is available in https://github.com/reach2ashish/geograpy, and can be installed by running:
# python3 -m pip install git+https://github.com/reach2ashish/geograpy.git

from reviews_data_standardized import preprocess_countries_reviews_table

# Import output
reviews_pre = preprocess_countries_reviews_table(reviews)
#reviews_pre = reviews_pre[reviews_pre['Longitude'] != '2013-07-01']
reviews_pre.drop_duplicates()
reviews_pre['Longitude'].astype('float')
print(list(reviews_pre.columns))
# Select columns
rev_df = reviews_pre[[ 'Property ID', 'User ID', 'Review Date', 'Member Since', 'First Name', 
                   'Country', 'State', 'City', 'Description', 'School', 'Work', 
                   'country_after_parse', 'country_from_city_parse', 'final_country_parse' ]]



Finally, this is the result of our preprocessing:

In [ ]:
# plot class variables
for column in ['country_after_parse', 'country_from_city_parse', 'final_country_parse']: #'Description', 'School', 'Work']:
    # plot bar chart
    rev_df[rev_df[column] != '-'][column].value_counts().nlargest(50).plot(kind='bar', figsize=(22, 6), title=column)
    #plt.set_title(column)
    if column == 'final_country_parse':
        plt.savefig(plt_viz+'final_country_parse.png')
    plt.show()
    print( 'Count of non-parsed origins: ', len(rev_df[rev_df[column] == '-'][column]) )

Comparing the final parse with the initial top 50 origins distribution:

In [ ]:

# plot bar chart
reviews[reviews['Country'] != '-']['Country'].value_counts().nlargest(50).plot(kind='bar', figsize=(22, 6), title='Country')
#plt.set_title(column)
plt.show()


** Finally, a sum up of all the analysis in all tables: **

In [ ]:
i=0
table_name = ['properties', 'daily', 'monthly', 'reviews']
for table in [properties, daily, monthly, reviews_pre]:
    print(table_name[i])
    i+=1
    analysis = table.describe().T
    analysis['na_count_percent'] = np.round(table.isnull().sum() / len(table) *100, 1)
    analysis['na_count_percent'] = analysis['na_count_percent'].astype('str')+'%'
    analysis = analysis.reset_index()
    columns = list(analysis.columns)
    columns[0] = 'columns'
    analysis.columns = columns
    
    trace = go.Table(
        header=dict(values=analysis.columns, fill = dict(color='#C2D4FF'), align = ['center'] * 5),
        cells=dict(values=[analysis[column] for column in analysis.columns],fill = dict(color='#F5F8FF'), 
                   align = ['left'] * 5))
    plotly.offline.iplot([trace], show_link=False)



### Missing Values Detection

In order to proceed to the next step of the analysis, we will need to handle missing values. As such, we will check for inexistent values in all variables from the 3 different tables, displayed in the previous set of tables.

In [ ]:
nan_props = properties.isnull().sum()
nan_daily = daily.isnull().sum()
nan_monthly = monthly.isnull().sum()
nan_reviews = reviews.isnull().sum()

# create dict with NaN table counts for loop
nans = dict(nan_props = nan_props,
            nan_daily = nan_daily,
            nan_monthly = nan_monthly,
            nan_reviews = nan_reviews)

# set dict with total size of data sets
lengths = dict(nan_props      = len(properties),
               nan_daily      = len(daily),
               nan_monthly    = len(monthly),
               nan_reviews    = len(reviews))

nan_tables = []
for nan_count in nans:
    nan_counts = pd.DataFrame(nans[nan_count], columns= [ 'nan_count' ]).reset_index()
    nan_counts.columns = ['columns','nan_count']
    # calculate percentage over the total size of the data set for relative analysis
    nan_counts['nan%'] = nan_counts['nan_count'].apply(lambda x: (x/lengths[nan_count])*100)
    nan_tables.append(nan_counts)

# configure table
final_nan = pd.concat(nan_tables, ignore_index=True, axis=1)
final_nan.columns = ['Properties', 'nan_p', 'nan%_p', 'Daily', 'nan_d', 'nan%_d', 
                     'Monthly', 'nan_m', 'nan%_m', 'Reviews', 'nan_r', 'nan%_r']

# replace NaN's where there are no columns to show
final_nan = final_nan.fillna(value='-')

# configure table viz
trace = go.Table(
    header=dict(values=final_nan.columns, fill = dict(color='#C2D4FF'), align = ['left'] * 5),
    cells=dict(values=[final_nan[column] for column in final_nan.columns],fill = dict(color='#F5F8FF'), 
               align = ['left'] * 5))
plotly.offline.iplot([trace], show_link=False)


Here we see that there are some columns that have a very high percentage of missing values, which we must consider whether to use them or not and how can we proceed to fill the missing values in the variables we will be using. To do this, we must consider each variable's importance to our goal: **Tourism profiling and flows analysis**.

Once again, given the size of the present datasets, we will use the median to fill the missing values in the necessary columns, i.e., the ones we will be using for the remainder of the analysis:

In [ ]:

# outliers will be handled in these tables
tables_filtered2 = dict(properties = properties,
                       daily      = daily,
                       monthly    = monthly,
                       reviews    = reviews_pre)

tables_filtered2['monthly']['Revenue (USD)'] = tables_filtered2['monthly']['Revenue (USD)'].fillna(value=0)

medians_props = tables_filtered2['properties'].median(skipna=True)
medians_monthly = tables_filtered2['monthly'].median(skipna=True)
medians_daily = tables_filtered2['daily'].median(skipna=True)
medians_reviews = tables_filtered2['reviews'].median(skipna=True)
tables_filtered2['properties'] = tables_filtered2['properties'].fillna(value=medians_props)
tables_filtered2['monthly'] = tables_filtered2['monthly'].fillna(value=medians_monthly)
tables_filtered2['daily'] = tables_filtered2['daily'].fillna(value=medians_daily)
tables_filtered2['reviews'] = tables_filtered2['reviews'].fillna(value=medians_reviews)

tables_filtered = tables_filtered2


**"the stats are inaccurate. The number of blocked days is wrong. The number of booked dates is wrong. The number of unbooked days is wrong. Clearly, AirBNB is not doing their own calculations correctly. Thankfully, the SuperHost numbers are correct."**
 
It is impossible to assess whether the number of reviews is representative, given that Airbnb's accuracy in their data is not certain. Furthermore, Airbnb's reviews can be either public or private. As we are only using publicly available data, we do not have access to user data that left a private review, or no review at all. So, we are analysing user profiles that represent 10% of the total bookings that were actually completed. Although the user data sample extracted from the overall reviews was not randomly generated, it is highly representative. The number of total completed bookings made between September 1st 2014 and December 31st 2017 is 11.550 million Bookings which implies a minimum sample size of 16564 for a 99% confidence level and 1% margin of error. As our sample has a size of 1.2 million observations, it is statistically significant (although, we cannot conclude that it is an unbiased sample, as it was not randomly selected).

### Outlier Detection

Aside from the excessively high revenue values in the table Property Listings, we did not detect any other value that we were certain to be wrong. So, we conclude that our dataset doesn't have values that require its removal for this reason. Although, we did find some outliers.

In order to maintain a simple, straightforward analysis, we will use the Interquartile Range method (IQR) to remove these extreme values:

1) **IQR = Q3 – Q1**

2) **If a data point is below Q1 – 1.5×IQR or above Q3 + 1.5×IQR, it is viewed as being too far from the central values to be reasonable.**

After this, we will have two options: Remove these observations, or set these values as missing and reapply the method used in the previous step.


In [ ]:

#tables_filtered['properties'] = tables_filtered['properties'][tables_filtered['properties']['Number of Bookings LTM']>5]

# Filtering Values between Q1-1.5IQR and Q3+1.5IQR
for table in list(tables_filtered.keys()):
    if table == 'daily':
        #print('      ',table)
        for column in list(tables_filtered[table].columns):
            if column in ['Price (USD)']:#'Available Days']:
                #print(column)
                #Q1 = tables_filtered[table][column].dropna().quantile(0.25)
                #Q3 = tables_filtered[table][column].dropna().quantile(0.75)
                #IQR = Q3 - Q1
                #_filtered = tables_filtered[table][ Q1-1.5*IQR < tables_filtered[table][column]]
                #filtered = _filtered[ _filtered[column] < Q3+1.5*IQR ]
                #tables_filtered[table] = filtered
                #print(len(tables_filtered[table]))
                tables_filtered[table] = tables_filtered[table][tables_filtered[table][column] < 400]
                
    elif table == 'monthly':
        column = 'ADR (USD)'
        condition = tables_filtered[table][column] < 2000.00
        tables_filtered[table] = tables_filtered[table][condition]
    
    elif table == 'properties':
        tables_filtered[table] = tables_filtered[table][tables_filtered[table]['Annual Revenue LTM (USD)'] < 100000]
    
    elif table == 'reviews':
        tables_filtered[table] = tables_filtered[table].drop_duplicates()

#tbls = list(tables_filtered.keys())

print('Number of observations filtered:')
print('properties',': ', (len(tables['properties'])-len(tables_filtered['properties'])))
print('daily',': ', (len(tables['daily'])-len(tables_filtered['daily'])))
print('monthly',': ', (len(tables['monthly'])-len(tables_filtered['monthly'])))
print('reviews',': ', (len(tables['reviews'])-len(tables_filtered['reviews'])))


____
____

## Modelling

To analyse patterns in the provided data we will extract as many relevant variables from customers as possible. Additionally, we will also segment Property Listings by geographic location and associate them to the customers.

### Property Listings Correlation Analysis

In [ ]:
properties_column_list = [column for column in tables_filtered['properties'].columns]
properties_column_list.remove('Zipcode')
properties_column_list.remove('Metropolitan Statistical Area')
properties_column_list.remove('Property ID')
properties_column_list.remove('Host ID')
properties_column_list.remove('Last Scraped Date')
properties_column_list.remove('Country')
properties_column_list.remove('State')
properties_column_list.remove('City')
properties_column_list.remove('Neighborhood')
properties_column_list.remove('Checkout Time')
properties_column_list.remove('Superhost')
properties_column_list.remove('Currency Native')
properties_column_list.remove('Listing Type')
properties_column_list.remove('Listing Title')
properties_column_list.remove('Property Type')
properties_column_list.remove('Cancellation Policy')
properties_column_list.remove('Calendar Last Updated')
properties_column_list.remove('Listing URL')
properties_column_list.remove('Business Ready')
properties_column_list.remove('Listing Main Image URL')
properties_column_list.remove('Longitude')
properties_column_list.remove('Latitude')

corr_matrix = tables_filtered['properties'][properties_column_list].reset_index().corr(method='pearson')


trace = go.Heatmap( z= corr_matrix.values.tolist(),
                    x= properties_column_list,
                    y= properties_column_list,
                  )
data = go.Data([trace])

layout = dict( margin = dict(t=50,r=50,b=150,l=150))

figure = dict(data=data , layout=layout )

plotly.offline.plot(figure, filename=viz_out+'correlation_matrix_test.html', show_link=False, auto_open=False)
plotly.offline.iplot(figure, show_link=False)

**(Insights taken from the correlation matrix goes here, and there are plenty)**

### Property Listings Geographic Clustering

In [ ]:

print('Opening shapefile with districts\' borders...')

shp_file = fiona.open(shp_dir+'PRT_adm2.shp')

distritos = pd.read_csv(shp_dir+'PRT_adm2.csv')[['ID_2', 'NAME_2']]
distritos = distritos.set_index('ID_2')

print('Formatting shapefile and defining polygons and points...')
shp_final = []
for distrito in shp_file:
    distrito.update({'shape':shape(distrito['geometry'])})
    shp_final.append(distrito)

def generate_point(row):
    return Point(row['Longitude'],row['Latitude'])

tables_filtered['properties']['point'] = tables_filtered['properties'].apply(generate_point, axis=1)


def check_district(row):
    for distrito in shp_final:
        check = distrito['shape'].contains(row['point'])
        if check:
            return distrito['id']

print('Assigning the respective district id\'s to the points...')
tables_filtered['properties']['cluster'] = tables_filtered['properties'].apply(check_district, axis=1)

print('Assigning district id\'s to points outside of borders (i.e., the ones with imprecisions in coordinates)...')
tables_filtered['properties']['cluster'] = tables_filtered['properties']['cluster'].fillna(30).astype('int')
n_neighbors = 3
X = tables_filtered['properties'][tables_filtered['properties']['cluster'] != 30][['Latitude', 'Longitude']]
y = tables_filtered['properties'][tables_filtered['properties']['cluster'] != 30]['cluster']
clf = neighbors.KNeighborsClassifier(n_neighbors)
clf.fit(X, y)
knn_prediction = clf.predict(tables_filtered['properties'][tables_filtered['properties']['cluster'] == 30][['Latitude', 'Longitude']])
tables_filtered['properties'].loc[tables_filtered['properties']['cluster'] == 30, 'cluster'] = knn_prediction

print('Assigning district names to id\'s')
def get_area_name(value):
    return distritos.loc[value+1]['NAME_2']


tables_filtered['properties']['cluster_2'] = tables_filtered['properties']['cluster']
tables_filtered['properties']['cluster'] = tables_filtered['properties']['cluster'].apply(get_area_name)


print('Done!')


In [ ]:

tables_filtered['properties'] = tables_filtered['properties'].sort_values('cluster')
map_data = tables_filtered['properties'][tables_filtered['properties']['Number of Bookings LTM']>5]

data = go.Data([
        go.Scattermapbox(
                lat = map_data['Latitude'],
                lon = map_data['Longitude'],
                mode='markers',
                marker=go.Marker(size= 2,
                                 color=map_data['cluster_2'] ),
                text = map_data['cluster']
                )
        ])

layout = go.Layout(
    title='Location of listings',
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_token,
        bearing=0,
        style='dark',
        center=dict(
            lat=39.64,
            lon=-7.95,
        ),
        pitch=0,
        zoom=4.2
    ),
)

wonder_map = go.Figure(data=data, layout=layout)

#generate interactive visualization
plotly.offline.plot(wonder_map, filename=viz_out+'geographic_clustering.html', show_link=False, auto_open=False)
plotly.offline.iplot(wonder_map, show_link=False)





### Property Listings' Value Clustering

In this clustering analysis we will cluster listings by economic and property features, as well as details in Airbnb platform such as the number of pictures in a listing.

In [ ]:
# select variables
filter_condition = tables_filtered['properties']['Number of Bookings LTM'] > 4
val = tables_filtered['properties'][filter_condition]\
                                   [['Property ID', 
                                     'Latitude', 
                                     'Longitude', 
                                     'Annual Revenue LTM (USD)', 
                                     'Average Daily Rate (USD)', 
                                     'Occupancy Rate LTM', 
                                     'Number of Bookings LTM',
                                     # 'Number of Reviews', <- highly correlated with number of bookings
                                     'Bedrooms', 
                                     'Bathrooms', 
                                     'Max Guests']]

val = val.fillna(val.mean())

# Normalization of the variables we will use for the value clustering
vars_ = ['Annual Revenue LTM (USD)', 'Average Daily Rate (USD)', 'Occupancy Rate LTM', 
                     'Number of Bookings LTM']
val[vars_] = preprocessing.scale(val[vars_])

#####################################################################################################################

inertia_table = pd.DataFrame(index=list(range(2,21)))

# estimating optimal number of clusters
for nmbr_of_runs in list(range(1,4)):
    
    distances = []
    for nmbr in list(range(2,21)):
        # Define K-means algorithm
        km = sk_cluster.KMeans(n_clusters=nmbr, random_state=None)
        km.fit( val[vars_] )
        # Get Cluster assignment Labels
        ine = km.inertia_
        distances.append(ine)
        
    inertia_table['inertia_iteration_'+str(nmbr_of_runs)] = distances

inertia_table.T.mean().plot.line()
plt.show()

#inertia_table.plot.line()
#plt.show()


In [ ]:


# Define K-means algorithm
km = sk_cluster.KMeans(n_clusters=5, random_state=None)
km.fit(val[['Annual Revenue LTM (USD)', 'Average Daily Rate (USD)', 'Occupancy Rate LTM', 'Number of Bookings LTM']])



# Get Cluster assignment Labels
labels = km.labels_
val['val_cluster'] = labels

data = go.Data([
        go.Scattermapbox(
                lat = val['Latitude'],
                lon = val['Longitude'],
                mode='markers',
                marker=go.Marker(size= 2,
                                 color=val['val_cluster'] ),
                text = val['val_cluster']
                )
        ])

layout = go.Layout(
    title='Location of listings',
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_token,
        bearing=0,
        style='dark',
        center=dict(
            lat=39.64,
            lon=-7.95,
        ),
        pitch=0,
        zoom=4.2
    ),
)

super_map = go.Figure(data=data, layout=layout)

#generate interactive visualization
plotly.offline.plot(super_map, filename=viz_out+'value_clustering.html', show_link=False, auto_open=False)
plotly.offline.iplot(super_map, show_link=False)


### Profiling

We will need to merge and adapt our data (group it by value cluster, region, month, etc) in order to analyse it further:

In [ ]:
# Select the tables we will use
rev = tables_filtered['reviews'][['User ID', 'Property ID', 'final_country_parse', 'Review Date']]
prop = tables_filtered['properties'][[ 'Property ID', 'Average Daily Rate (USD)', 'Listing Type', 'Bedrooms', 'cluster' ]]
day = tables_filtered['daily'][tables_filtered['daily']['Status'] == 'R'][['Property ID', 'Booked Date', 'Date', 'Price (USD)']]
mon = tables_filtered['monthly'][[ 'Property ID', 'Reporting Month', 'Occupancy Rate', 'ADR (USD)', 'Number of Reservations', 'Revenue (USD)', 'Reservation Days' ]]

# Merge tables into 2
tourists = pd.merge(rev, prop, how= 'left', on= 'Property ID')
locations = pd.merge(mon, prop, how = 'left', on= 'Property ID')

# Format columns
tourists['Review Date'] = pd.to_datetime(tourists['Review Date'])
locations['Reporting Month'] = pd.to_datetime(locations['Reporting Month'])
locations['Review Date'] = locations['Reporting Month']


In [ ]:
tourists['Listing Type'] = tourists['Listing Type'].fillna('unknown')


# define class variables by 1 or 0's for each column to determine percentages afterwards
def get_cluster(value, cluster):
    if value == cluster:
        ones = 1
    else:
        ones = 0
    return ones

# apply function to the class type columns
cluster_nums = list(tourists.sort_values('cluster')['cluster'].dropna().unique())
#cluster_nums.append(1000)
cluster_nums_columns = ['cluster_'+str(cluster) for cluster in cluster_nums]
for cluster in cluster_nums:
    column_name = 'cluster_'+str(cluster)
    tourists[column_name] = tourists['cluster'].apply(lambda x: get_cluster(x, cluster))

# apply function to the class type columns
property_type_index = list(tourists['Listing Type'].unique())
property_type_columns = [ 'listing_type_'+str(listing) for listing in property_type_index]
for listing_type in property_type_index:
    column_name = 'listing_type_'+str(listing_type)
    tourists[column_name] = tourists['Listing Type'].apply(lambda x: get_cluster(x, listing_type))

# apply function to the class type columns
origins_index = list(tourists['final_country_parse'].value_counts().nlargest(30).keys())
#origins_index.remove('French Southern Territories')
#origins_index.remove('Saint Helena, Ascension and Tristan da Cunha')
#origins_index.remove('Lao People\'s Democratic Republic')
for origin in origins_index:
    tourists[origin] = tourists['final_country_parse'].apply(lambda x: get_cluster(x, origin))


# generate final tables for visualization
origins_grouped = tourists.drop(origins_index, axis=1).groupby(['final_country_parse', 'Review Date']).mean()
origins_grouped['review_count'] = tourists.groupby(['final_country_parse', 'Review Date']).size()

regions_grouped = tourists.drop(cluster_nums_columns, axis=1).groupby(['cluster', 'Review Date']).mean()
regions_grouped['review_count'] = tourists.groupby(['cluster', 'Review Date']).size()
regions_grouped = regions_grouped.reset_index()

pre_merg = locations[[ 'cluster', 'Review Date', 'Occupancy Rate', 'Number of Reservations', 'Revenue (USD)', 'Bedrooms' ]]
pre_merg1 = pre_merg.groupby(['cluster', 'Review Date']).mean()
pre_merg2 = pre_merg.groupby(['cluster', 'Review Date']).sum()
pre_merg3 = pre_merg1[[ 'Occupancy Rate' ]]
pre_merg3['Sum of Reservations'] = pre_merg2['Number of Reservations']
pre_merg3['projected_revenue'] = pre_merg2['Revenue (USD)']
pre_merg3['Avg Occupancy Rate'] = pre_merg3['Occupancy Rate']
final_merg = pre_merg3[[ 'Sum of Reservations', 'Avg Occupancy Rate', 'projected_revenue' ]].reset_index()

regions_grouped = pd.merge(regions_grouped, final_merg, on=['cluster', 'Review Date'], how='left')

# overall instead of regions
overall_grouped = tourists.drop(cluster_nums_columns, axis=1).groupby(['Review Date']).mean()
overall_grouped['review_count'] = tourists.groupby(['Review Date']).size()
overall_grouped = overall_grouped.reset_index()

ovrl_pre_merg = locations[['Review Date', 'Occupancy Rate', 'Number of Reservations', 'Revenue (USD)' ]]
ovrl_pre_merg1 = ovrl_pre_merg.groupby(['Review Date']).mean()
ovrl_pre_merg2 = ovrl_pre_merg.groupby(['Review Date']).sum()
ovrl_pre_merg3 = ovrl_pre_merg1[[ 'Occupancy Rate' ]]
ovrl_pre_merg3['Sum of Reservations'] = ovrl_pre_merg2['Number of Reservations']
ovrl_pre_merg3['projected_revenue'] = ovrl_pre_merg2['Revenue (USD)']
ovrl_pre_merg3['Avg Occupancy Rate'] = ovrl_pre_merg3['Occupancy Rate']
ovrl_final_merg = ovrl_pre_merg3[[ 'Sum of Reservations', 'Avg Occupancy Rate', 'projected_revenue' ]].reset_index()

overall_grouped = pd.merge(overall_grouped, ovrl_final_merg, on=['Review Date'], how='left')




In [ ]:
# export tables
regions_grouped.to_csv(preprocess_dir+'portuguese_regions_clusters.csv')
origins_grouped.to_csv(preprocess_dir+'countries_of_origin.csv')


In [ ]:
trace_list = []
for cluster in cluster_nums:    
    trace = {
        "x": regions_grouped[regions_grouped['cluster'] == cluster]['Review Date'], 
        "y": regions_grouped[regions_grouped['cluster'] == cluster]['Sum of Reservations'],
        "mode": "lines", 
        "name": cluster,
        "text": cluster,
        }
    trace_list.append(trace)


data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "Number of reservations per cluster", 
  "xaxis": {
    "autorange": False, 
    "range": ["2014-08-31 15:42:21.1765", "2017-12-04 18:52:56.4706"], 
    "title": "Date", 
    "type": "date"
  }, 
  "yaxis": {
    "autorange": True, 
#    "range": [-5791.0, 110029], 
    "title": "Sum of Reservations", 
    "type": "linear"
  }
}


fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'reservation_growth_per_region.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)

In [ ]:
trace_list = []
for cluster in cluster_nums:    
    trace = {
        "x": regions_grouped[regions_grouped['cluster'] == cluster]['Review Date'], 
        "y": regions_grouped[regions_grouped['cluster'] == cluster]['projected_revenue'], 
        "mode": "lines", 
        "name": cluster,
        "text": cluster,
        }
    trace_list.append(trace)

data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "Revenue per cluster", 
  "xaxis": {
    "autorange": False, 
    "range": ["2014-08-31 15:42:21.1765", "2017-12-04 18:52:56.4706"], 
    "title": "Date", 
    "type": "date"
  }, 
  "yaxis": {
    "autorange": True, 
#    "range": [-5791.0, 110029], 
    "title": "Sum of Revenue", 
    "type": "linear"
  }
}


fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'revenue_growth_per_region.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)

In [ ]:
trace_list = []

for cluster in cluster_nums:
    trace = {
        "x": regions_grouped[regions_grouped['cluster'] == cluster]['Review Date'], 
        "y": regions_grouped[regions_grouped['cluster'] == cluster]['projected_revenue'] / regions_grouped[regions_grouped['cluster'] == cluster]['Sum of Reservations']
        , 
        "mode": "lines", 
        "name": cluster,
        "text": cluster,
        }
    trace_list.append(trace)

data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "Revenue per reservation", 
  "xaxis": {
    "autorange": False, 
    "range": ["2016-03-30 15:42:21.1765", "2017-12-04 18:52:56.4706"], 
    "title": "Date", 
    "type": "date"
  }, 
  "yaxis": {
    "autorange": False, 
    "range": [0, 1000], 
    "title": "Revenue/Reservations Ratio", 
    "type": "linear"
  }
}

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'revenue_per_reservation_per_region.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)

In [ ]:

prop2 = tables_filtered['properties'][tables_filtered['properties']['Number of Bookings LTM']>4][[ 'Property ID', 'Average Daily Rate (USD)', 'Listing Type', 'Bedrooms', 'Created Date', 'cluster']]

daterange = pd.date_range('2009-10-01','2018-01-01' , freq='1M') 
daterange = daterange.union([daterange[-1] + 1])  
daterange = [date.replace(day=1).strftime('%Y-%m-%d') for date in daterange]


df_list = []
for cluster in cluster_nums:
    df1 = pd.DataFrame(daterange)
    df1['date'] = df1[0]
    df1['cluster'] = cluster
    df_list.append(df1[['cluster','date']])

indexed = df_list[0]
for df__ in list(range(1,len(df_list))):
    indexed = indexed.append(df_list[df__], ignore_index=True)


df_for_calcs = prop2[['Created Date', 'cluster']].dropna()
df_for_calcs['Bedrooms'] = prop2['Bedrooms']
indexed['date'] = pd.to_datetime(indexed['date'])
df_for_calcs['date'] = pd.to_datetime(df_for_calcs['Created Date'])

indexed['bedroom_count'] = 0
indexed['property_count'] = 0
for row in range(len(indexed)):
    cluster = indexed['cluster'][row]
    current_time = indexed['date'][row]
    cluster_filter = df_for_calcs[df_for_calcs['cluster']==cluster]
    time_filter = cluster_filter[cluster_filter['date'] < current_time]
    indexed['property_count'][row] = len(time_filter)
    indexed['bedroom_count'][row] = time_filter.sum()['Bedrooms']


supply_demand_table = regions_grouped[['cluster', 'Review Date', 'Sum of Reservations', 'projected_revenue']]
supply_demand_table['date'] = supply_demand_table['Review Date']
supply_demand_table[['cluster', 'date', 'Sum of Reservations', 'projected_revenue']]

supply_demand_table = pd.merge(supply_demand_table, indexed, on=['cluster', 'date'], how='left')

supply_demand_table.dropna()


# plot viz
trace_list = []
for cluster in cluster_nums:
    table_filtered__ = supply_demand_table[supply_demand_table['cluster'] == cluster]
    trace = {
        "x": table_filtered__['date'], 
        "y": table_filtered__['Sum of Reservations'] / table_filtered__['property_count']
        , 
        "mode": "lines", 
        "name": cluster,
        "text": cluster,
        }
    trace_list.append(trace)

data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "Demand/Supply Ratio", 
  "xaxis": {
    "autorange": False, 
    "range": ["2014-08-31 15:42:21.1765", "2017-12-04 18:52:56.4706"], 
    "title": "Date", 
    "type": "date"
  }, 
  "yaxis": {
    "autorange": True, 
#    "range": [-5791.0, 110029], 
    "title": "Reservations/Properties Ratio", 
    "type": "linear"
  }
}

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'demand_supply_plot.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)




In [ ]:
"""
pies = []
domains = [
    {'y': [0.8, 1-0.03], 'x': [0, 0.25-0.02]},
    {'y': [0.8, 1-0.03], 'x': [0.25, 0.5-0.02]},
    {'y': [0.8, 1-0.03], 'x': [0.5, 0.75-0.02]},
    {'y': [0.8, 1-0.03], 'x': [0.75, 1-0.02]},
    {'y': [0.6, 0.8-0.03], 'x': [0, 0.25-0.02]},
    {'y': [0.6, 0.8-0.03], 'x': [0.25, 0.5-0.02]},
    {'y': [0.6, 0.8-0.03], 'x': [0.5, 0.75-0.02]},
    {'y': [0.6, 0.8-0.03], 'x': [0.75, 1-0.02]},
    {'y': [0.4, 0.6-0.03], 'x': [0, 0.25-0.02]},
    {'y': [0.4, 0.6-0.03], 'x': [0.25, 0.5-0.02]},
    {'y': [0.4, 0.6-0.03], 'x': [0.5, 0.75-0.02]},
    {'y': [0.4, 0.6-0.03], 'x': [0.75, 1-0.02]},
    {'y': [0.2, 0.4-0.03], 'x': [0, 0.25-0.02]},
    {'y': [0.2, 0.4-0.03], 'x': [0.25, 0.5-0.02]},
    {'y': [0.2, 0.4-0.03], 'x': [0.5, 0.75-0.02]},
    {'y': [0.2, 0.4-0.03], 'x': [0.75, 1-0.02]},
    {'y': [0, 0.2-0.03], 'x': [0, 0.25-0.02]},
    {'y': [0, 0.2-0.03], 'x': [0.25, 0.5-0.02]},
    {'y': [0, 0.2-0.03], 'x': [0.5, 0.75-0.02]},
    {'y': [0, 0.2-0.03], 'x': [0.75, 1-0.02]},
    ]

x_coordinates_ = [0.115, 0.25+0.115, 0.5+0.115, 0.75+0.115]
y_coordinates_ = [0.99, 0.79, 0.58, 0.385, 0.17]
x_coordinates = x_coordinates_
y_coordinates = []
for i in range(0,4):
    x_coordinates = x_coordinates + x_coordinates_
    y_coordinates = y_coordinates + y_coordinates_

y_coordinates = sorted(y_coordinates, reverse=True)

annotations_coords = []
i=0
for cluster in cluster_nums:
    origins_per_cluster = tourists[['cluster', 'Review Date', 'final_country_parse']]
    filter_ = origins_per_cluster['cluster'] == cluster
    filter2 = origins_per_cluster['final_country_parse'] != '-'
    vals = origins_per_cluster[filter_][filter2][['final_country_parse', 'cluster']].groupby('final_country_parse').count()

    other_count = vals['cluster'].sum() - vals.nlargest(15, 'cluster')['cluster'].sum()

    pie_data = vals.nlargest(10, 'cluster' )
    pie_data.loc['others'] = [other_count]

    labels = list(pie_data.index)
    values = list(pie_data['cluster'])
    name = cluster
    
    a_nice_pie = go.Pie(values=values, 
                        labels=labels, 
                        name=name,
                        domain=domains[i],
                        textinfo="none",
#                        hole = .4
                       )
    
    pies.append(a_nice_pie)

    annotations=go.Annotation(
                x=x_coordinates[i],
                y=y_coordinates[i],
                showarrow=False,
                xanchor='center',
#                xref='paper',
#                yref='paper',
                align='center',
                text=cluster,
#                ax=0,
#                ay=-40
                )
    annotations_coords.append(annotations)
    i+=1


layout = go.Layout(autosize = True,
                   showlegend=True,
                   #width=600,
                   height=1000,
                   annotations=go.Annotations(annotations_coords),
                   title = 'Nationality Representation For Each Region')

fig = go.Figure(data = pies, layout = layout)

plotly.offline.plot(fig, filename=viz_out+'pies_nacionality_per_region.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)
"""

In [ ]:


origins_grouped_ovrl = tourists.drop(origins_index, axis=1).groupby(['final_country_parse']).mean()
origins_grouped_ovrl['review_count'] = tourists.groupby(['final_country_parse']).size()

origins_grouped_ovrl.columns

origins_grouped_ovrl = origins_grouped_ovrl[[ 'review_count', 'Average Daily Rate (USD)', 'Bedrooms',# 'cluster_Aveiro',
                                              #'cluster_Açores', 'cluster_Beja', 'cluster_Braga', 'cluster_Bragança',
                                              #'cluster_Castelo Branco', 'cluster_Coimbra', 'cluster_Evora',
                                              #'cluster_Faro', 'cluster_Guarda', 'cluster_Leiria', 'cluster_Lisboa',
                                              #'cluster_Madeira', 'cluster_Portalegre', 'cluster_Porto',
                                              #'cluster_Santarem', 'cluster_Setubal', 'cluster_Viana do Castelo',
                                              #'cluster_Vila Real', 'cluster_Viseu',
                                              'listing_type_Entire home/apt', 'listing_type_Private room',
                                              'listing_type_unknown', 'listing_type_Shared room' ]]

filter_ = origins_grouped_ovrl['review_count'] > 1000
origins_grouped_ovrl = origins_grouped_ovrl[filter_]
"""
origins_grouped_ovrl['most_visited_cluster'] = '-'
origins_grouped_ovrl['second_most_visited'] = '-'
origins_grouped_ovrl['third_most_visited'] = '-'
for row in range(len(origins_grouped_ovrl)):
    highest_value = 0
    second_highest_value = 0
    third_highest_value = 0
    
    highest_i = 0
    second_highest_i = 0
    third_highest_i = 0
    
    for i in range(0,10):
        current_value = origins_grouped_ovrl['cluster_'+str(i)][row]
        if current_value > third_highest_value :
            if current_value > second_highest_value:
                if current_value > highest_value:
                    third_highest_i = second_highest_i
                    second_highest_i = highest_i
                    highest_i = i
                    
                    third_highest_value = second_highest_value
                    second_highest_value = highest_value
                    highest_value = current_value
                else:
                    third_highest_i = second_highest_i
                    second_highest_i = i
                    
                    third_highest_value = second_highest_value
                    second_highest_value = current_value
            else:
                third_highest_value = current_value
                
                third_highest_i = i
    
    origins_grouped_ovrl['most_visited_cluster'][row] = highest_i
    origins_grouped_ovrl['second_most_visited'][row] = second_highest_i
    origins_grouped_ovrl['third_most_visited'][row] = third_highest_i
"""
origins_grouped_ovrl.nlargest(30, 'review_count' )

In [ ]:
trace_data = origins_grouped_ovrl.loc[origins_grouped_ovrl.index != '-'].sort_values('Average Daily Rate (USD)', ascending=False)

trace1 = {
  "x": list(trace_data.index),
  "y": list(trace_data['Average Daily Rate (USD)']), 
  "marker": {"color": "rgb(151, 160, 160)"}, 
  "name": "Average Daily Rate (USD)", 
  "type": "bar", 
  "uid": "247a10", 
  "xsrc": "qiweihan:653:a556e9", 
  "ysrc": "qiweihan:653:582f23",
  "text": ["# of reviews: "+str(rate) for rate in trace_data['review_count']]
}
data = go.Data([trace1])
layout = {
  "autosize": True, 
  "dragmode": "lasso", 
  "hovermode": "x", 
  "title": "Average Daily Rates for each origin", 
  "xaxis": {
    "autorange": True, 
    "range": [-0.5, 56.5], 
    "showspikes": False, 
    "title": "", 
    "type": "category"
  }, 
  "yaxis": {
    "autorange": True, 
    "range": [0, 106.383682389], 
    "showspikes": False, 
    "title": "Avg Daily Rate", 
    "type": "linear"
  }
}
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'ADR_origins.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)





In [ ]:

ldf = locations[['Property ID', 'cluster','Review Date', 
                 'Occupancy Rate','Number of Reservations', 'Reservation Days']]

tdf = tourists[['Property ID', 'Review Date', 'final_country_parse' ]]

def estimated_avg_stay(row):
    try:
        return row['Reservation Days']/row['Number of Reservations']
    except ZeroDivisionError:
        return 0

ldf['estimated_avg_stay'] = ldf.apply(lambda x: estimated_avg_stay(x), axis=1)

# formatting data for length of stay per country of origin
stay_estimate_origin = pd.merge(tdf, ldf, how='left', on=['Property ID','Review Date']) \
                        [[ 'final_country_parse', 'Review Date', 'estimated_avg_stay' ]]

stay_estimate_origin_grouped = stay_estimate_origin[[ 'final_country_parse','estimated_avg_stay' ]] \
                                .groupby(['final_country_parse']).mean().reset_index()

review_count = tdf.groupby('final_country_parse').size().to_frame(name='review_count').reset_index()

stay_estimate_origin_grouped = pd.merge(stay_estimate_origin_grouped, review_count, how='left' , on='final_country_parse' ) \
                                .nlargest(50,'review_count')

stay_estimate_origin_grouped = stay_estimate_origin_grouped.sort_values('estimated_avg_stay', ascending=False)

# formatting data for length of stay per region
stay_estimate_country = pd.merge(tdf, ldf, how='left', on=['Property ID','Review Date']) \
                        [[ 'cluster', 'Review Date', 'estimated_avg_stay' ]]

stay_estimate_country_grouped = stay_estimate_country[['cluster', 'estimated_avg_stay']].groupby('cluster').mean()\
                                .reset_index()



In [ ]:
trace_data_2 = stay_estimate_origin_grouped[stay_estimate_origin_grouped['final_country_parse'] != '-']

trace1 = {
  "x": list(trace_data_2['final_country_parse']),
  "y": list(trace_data_2['estimated_avg_stay']), 
  "marker": {"color": "rgb(151, 160, 160)"}, 
  "name": "Estimated Average Stay", 
  "type": "bar", 
  "uid": "247a10", 
  "xsrc": "qiweihan:653:a556e9", 
  "ysrc": "qiweihan:653:582f23"
}
data = go.Data([trace1])
layout = {
  "autosize": True, 
  "dragmode": "lasso", 
  "hovermode": "x", 
  "title": "Average Length of stay for each origin", 
  "xaxis": {
    "autorange": True, 
    "range": [-0.5, 56.5], 
    "showspikes": False, 
    "title": "", 
    "type": "category"
  }, 
  "yaxis": {
    "autorange": True, 
    "range": [0, 106.383682389], 
    "showspikes": False, 
    "title": "Estimated Average Stay (days)", 
    "type": "linear"
  }
}
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'estimated_lengths_of_stay_origins.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)



In [ ]:
stay_estimate_country_grouped = stay_estimate_country_grouped.sort_values('estimated_avg_stay', ascending=False)

trace1 = {
  "x": list(stay_estimate_country_grouped['cluster']),
  "y": list(stay_estimate_country_grouped['estimated_avg_stay']), 
  "marker": {"color": "rgb(151, 160, 160)"}, 
  "name": "Estimated Average Stay", 
  "type": "bar", 
  "uid": "247a10", 
  "xsrc": "qiweihan:653:a556e9", 
  "ysrc": "qiweihan:653:582f23"
}
data = go.Data([trace1])
layout = {
  "autosize": True, 
  "dragmode": "lasso", 
  "hovermode": "x", 
  "title": "Average Length of stay for each region", 
  "xaxis": {
    "autorange": True, 
    "range": [-0.5, 56.5], 
    "showspikes": False, 
    "tickangle":45,
    "title": "", 
    "type": "category"
  }, 
  "yaxis": {
    "autorange": True, 
    "range": [0, 106.383682389], 
    "showspikes": False, 
    "title": "Estimated Average Stay (days)", 
    "type": "linear"
  }
}
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'estimated_lengths_of_stay_regions.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)



In [ ]:


price_variation = locations[['cluster', 'Reporting Month', 'Reservation Days', 'Revenue (USD)', 'ADR (USD)']]
#price_variation['estimated_daily_revenue'] = price_variation['Revenue (USD)'] / price_variation['Reservation Days']

price_variation = price_variation.groupby(['cluster', 'Reporting Month']).mean().reset_index()
price_variation_2 = locations[['cluster', 'Reporting Month', 'ADR (USD)', 'Number of Reservations']]
price_variation_2['reservations_adr_multiplied'] = price_variation_2['ADR (USD)']*price_variation_2['Number of Reservations']
price_variation_2['sum_of_reservations'] = price_variation_2['Number of Reservations']
price_variation_2 = price_variation_2.groupby(['cluster', 'Reporting Month']).sum().reset_index()
price_variation_2['weighted_adr'] = price_variation_2['reservations_adr_multiplied']/price_variation_2['sum_of_reservations']


trace_list = []
for cluster in cluster_nums:
    filter_condition = price_variation['cluster'] == cluster
    trace = {
        "x": price_variation_2[filter_condition]['Reporting Month'], 
        "y": price_variation_2[filter_condition]['weighted_adr'], 
        "mode": "lines", 
        "name": cluster,
        "text": cluster,
        }
    trace_list.append(trace)

data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "Weighted Average Daily Rate per region", 
  "xaxis": {
    "autorange": False, 
    "range": ["2015-12-31 15:42:21.1765", "2017-12-04 18:52:56.4706"], 
    "title": "Date", 
    "type": "date"
  }, 
  "yaxis": {
    "autorange": False, 
    "range": [0, 220], 
    "title": "Weighted ADR", 
    "type": "linear"
  }
}

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'estimated_adr_per_region.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)


In [ ]:


trace_list = []
for cluster in cluster_nums:    
    trace = {
        "x": regions_grouped[regions_grouped['cluster'] == cluster]['Review Date'], 
        "y": regions_grouped[regions_grouped['cluster'] == cluster]['Avg Occupancy Rate'], 
        "mode": "lines", 
        "name": cluster,
        "text": cluster,
        }
    trace_list.append(trace)

data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "Average Occupancy Rate per region", 
  "xaxis": {
    "autorange": False, 
    "range": ["2014-08-31 15:42:21.1765", "2017-12-04 18:52:56.4706"], 
    "title": "Date", 
    "type": "date"
  }, 
  "yaxis": {
    "autorange": True, 
#    "range": [-5791.0, 110029], 
    "title": "Avg Occupancy Rate", 
    "type": "linear"
  }
}


fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'occupancy_rate_per_region.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)

In [ ]:
origins_per_cluster = tourists[['Review Date', 'final_country_parse']]
origins_per_cluster_1 = origins_per_cluster.groupby(by=['final_country_parse']).size().to_frame(name='count').reset_index()
origins_per_cluster_1 = origins_per_cluster_1[origins_per_cluster_1['final_country_parse'] != '-'].nlargest(14, 'count')
countries = list(origins_per_cluster_1['final_country_parse'])

def top_countries_filter(country):
    if country not in countries:
        country = 'others'
    return country

origins_per_cluster_2 = origins_per_cluster
origins_per_cluster_2 = origins_per_cluster_2[origins_per_cluster_2['final_country_parse'] != '-']
origins_per_cluster_2['final_country_parse'] = origins_per_cluster_2['final_country_parse'].apply(top_countries_filter) 
origins_per_cluster_2 = origins_per_cluster_2.groupby(by=['Review Date', 'final_country_parse']).size().to_frame(name='count')
origins_per_cluster_2 = origins_per_cluster_2.reset_index()

origins_per_cluster_2

trace_list = []
countries.append('others')
for country in countries:
    filter__ = origins_per_cluster_2['final_country_parse'] == country
    trace = {
        "x": origins_per_cluster_2[filter__]['Review Date'], 
        "y": origins_per_cluster_2[filter__]['count'], 
        "mode": "lines", 
        "name": country,
        "text": country,
        }
    trace_list.append(trace)

data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "Count of booking reviews per Nationality", 
  "xaxis": {
    "autorange": False, 
    "range": ["2014-08-31 15:42:21.1765", "2017-12-04 18:52:56.4706"], 
    "title": "Date", 
    "type": "date"
  }, 
  "yaxis": {
    "autorange": True, 
#    "range": [-5791.0, 110029], 
    "title": "Count", 
    "type": "linear"
  }
}


fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'count_of_reviews_per_nationality.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)

## Value clustering analysis

In [ ]:
# Select the tables we will use
rev = tables_filtered['reviews'][['User ID', 'Property ID', 'final_country_parse', 'Review Date']]
prop = pd.merge(tables_filtered['properties'][[ 'Property ID', 'Average Daily Rate (USD)', 'Listing Type', 'Bedrooms', 'cluster' ]], val[['Property ID', 'val_cluster']], on='Property ID', how='left')
day = tables_filtered['daily'][tables_filtered['daily']['Status'] == 'R'][['Property ID', 'Booked Date', 'Date', 'Price (USD)']]
mon = tables_filtered['monthly'][[ 'Property ID', 'Reporting Month', 'Occupancy Rate', 'ADR (USD)', 'Number of Reservations', 'Revenue (USD)', 'Reservation Days' ]]

# Merge tables into 2
tourists = pd.merge(rev, prop, how= 'left', on= 'Property ID')
locations = pd.merge(mon, prop, how = 'left', on= 'Property ID')

# Format columns
tourists['Review Date'] = pd.to_datetime(tourists['Review Date'])
locations['Reporting Month'] = pd.to_datetime(locations['Reporting Month'])
locations['Review Date'] = locations['Reporting Month']

In [ ]:
ldfv = locations[[ 'val_cluster',
                'Occupancy Rate', 
                'ADR (USD)',
                'Number of Reservations', 
                'Revenue (USD)', 
                'Reservation Days',
                'Listing Type', 
                'Bedrooms']]


value_profiling = ldfv.groupby('val_cluster').mean().reset_index()
value_frequency = prop.groupby('val_cluster').size().to_frame(name='frequency').reset_index()
value_profiling = pd.merge(value_frequency, value_profiling, on='val_cluster')
#value_profiling

def round_decimals(value):
    return np.round(value, decimals=3)

# configure table viz
trace = go.Table(
    header=dict(values=value_profiling.columns, fill = dict(color='#ea7870'), align = ['left'] * 5),
    cells=dict(values=[value_profiling[column].apply(round_decimals) for column in value_profiling.columns], fill= dict(color='#efd9d7'), 
               align = ['left'] * 5))
plotly.offline.plot([trace], filename=viz_out+'value_clusters_profiling.html', show_link=False, auto_open=False)
plotly.offline.iplot([trace], show_link=False)


In [ ]:

trace_list = []
val_cluster_list = list(val.sort_values('val_cluster')['val_cluster'].unique())
for cluster in val_cluster_list:
    cluster_select = val['val_cluster'] == cluster
    trace = go.Scattermapbox(
                lat = val[cluster_select]['Latitude'],
                lon = val[cluster_select]['Longitude'],
                mode='markers',
                name="Cluster "+str(cluster),
                marker=go.Marker(size= 2,
                                 color=cluster ),
                text = val[cluster_select]['val_cluster']
                )
    trace_list.append(trace)



data = go.Data(trace_list)

layout = go.Layout(
    title='Location of listings',
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_token,
        bearing=0,
        style='dark',
        center=dict(
            lat=39.64,
            lon=-7.95,
        ),
        pitch=0,
        zoom=4.2
    ),
)

super_sub_map = go.Figure(data=data, layout=layout)
plotly.offline.plot(super_sub_map, filename=viz_out+'value_clustering.html', show_link=False, auto_open=False)

plotly.offline.iplot(super_sub_map, show_link=False)


In [ ]:

overall_indexed = indexed[['date', 'property_count', 'bedroom_count']].groupby('date').sum().reset_index()


overall_revpar_table = overall_grouped[['Review Date', 'Sum of Reservations', 'projected_revenue']]
overall_revpar_table['date'] = overall_revpar_table['Review Date']
overall_revpar_table = overall_revpar_table[['date', 'Sum of Reservations', 'projected_revenue']]

overall_revpar_table = pd.merge(overall_revpar_table, overall_indexed, on=['date'], how='left')

overall_revpar_table.dropna()


overall_revpar_table['revpar'] = overall_revpar_table['projected_revenue']/overall_revpar_table['bedroom_count']/30
supply_demand_table['revpar'] = supply_demand_table['projected_revenue']/supply_demand_table['bedroom_count']/30

trace_list = []
for cluster in cluster_nums:    
    trace = {
        "x": supply_demand_table[supply_demand_table['cluster'] == cluster]['date'], 
        "y": supply_demand_table[supply_demand_table['cluster'] == cluster]['revpar'],
        "mode": "lines", 
        "name": cluster,
        "text": cluster,
        }
    trace_list.append(trace)

# total revpar without region distinction
trace = {
        "x": overall_revpar_table['date'], 
        "y": overall_revpar_table['revpar'],
        "mode": "lines", 
        "name": 'Total',
        "text": 'Total',
        }
trace_list.append(trace)

data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "RevPAR", 
  "xaxis": {
    "autorange": False, 
    "range": ["2014-08-31 15:42:21.1765", "2018-01-15 18:52:56.4706"], 
    "title": "Date", 
    "type": "date"
  }, 
  "yaxis": {
    "autorange": True, 
#    "range": [-5791.0, 110029], 
    "title": "RevPAR", 
    "type": "linear"
  }
}


fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'Revpar_per_cluster_and_date.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)

In [ ]:
# yearly RevPAR

supply_demand_table['Year'] = supply_demand_table['Review Date'].dt.year

yearly_revpar_overall = supply_demand_table[['Year', 'projected_revenue', 'bedroom_count']].groupby('Year').sum().reset_index()
yearly_revpar_overall['revpar'] = yearly_revpar_overall['projected_revenue']/yearly_revpar_overall['bedroom_count']/30

yearly_revpar_clustered = supply_demand_table[['cluster','Year', 'projected_revenue', 'bedroom_count']] \
        .groupby(['Year','cluster']).sum().reset_index()

yearly_revpar_clustered['revpar'] = yearly_revpar_clustered['projected_revenue']/yearly_revpar_clustered['bedroom_count']/30


trace_list = []
for cluster in cluster_nums:    
    trace = {
        "x": yearly_revpar_clustered[yearly_revpar_clustered['cluster'] == cluster]['Year'].astype('int'), 
        "y": yearly_revpar_clustered[yearly_revpar_clustered['cluster'] == cluster]['revpar'],
        "mode": "lines", 
        "name": cluster,
        "text": cluster,
        }
    trace_list.append(trace)

# total revpar without region distinction
trace = {
        "x": yearly_revpar_overall['Year'].astype('int'), 
        "y": yearly_revpar_overall['revpar'],
        "mode": "lines", 
        "name": 'Total',
        "text": 'Total',
        }
trace_list.append(trace)

data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "RevPAR", 
  "xaxis": {
    "autorange": True, 
    #"range": ["2014-08-31 15:42:21.1765", "2018-01-15 18:52:56.4706"], 
    "title": "Date", 
    #"type": "date"
  }, 
  "yaxis": {
    "autorange": True, 
#    "range": [-5791.0, 110029], 
    "title": "RevPAR", 
    "type": "linear"
  }
}


fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'yearly_revpar.html', show_link=False, auto_open=True)
plotly.offline.iplot(fig, show_link=False)

In [ ]:

yearly_occupancy_rate = locations[locations['Number of Reservations'] > 0][['Review Date', 'cluster', 'Occupancy Rate']]
yearly_occupancy_rate['Year'] = yearly_occupancy_rate['Review Date'].dt.year

yearly_occupancy_rate = yearly_occupancy_rate[['Year', 'cluster', 'Occupancy Rate']] \
                                .groupby(['Year', 'cluster']).mean().reset_index()

yearly_occupancy_rate_overall = locations[['Review Date', 'cluster', 'Occupancy Rate']]
yearly_occupancy_rate_overall['Year'] = yearly_occupancy_rate_overall['Review Date'].dt.year

yearly_occupancy_rate_overall = yearly_occupancy_rate_overall[['Year', 'cluster', 'Occupancy Rate']] \
                                .groupby(['Year']).mean().reset_index()



trace_list = []
for cluster in cluster_nums:    
    trace = {
        "x": yearly_occupancy_rate[yearly_occupancy_rate['cluster'] == cluster]['Year'].astype('int'), 
        "y": yearly_occupancy_rate[yearly_occupancy_rate['cluster'] == cluster]['Occupancy Rate'],
        "mode": "lines", 
        "name": cluster,
        "text": cluster,
        }
    trace_list.append(trace)

# total revpar without region distinction
trace = {
        "x": yearly_occupancy_rate_overall['Year'].astype('int'), 
        "y": yearly_occupancy_rate_overall['Occupancy Rate'],
        "mode": "lines", 
        "name": 'Total',
        "text": 'Total',
        }
trace_list.append(trace)

data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "Occupancy Rate", 
  "xaxis": {
    "autorange": True, 
    #"range": ["2014-08-31 15:42:21.1765", "2018-01-15 18:52:56.4706"], 
    "title": "Date", 
    #"type": "date"
  }, 
  "yaxis": {
    "autorange": True, 
#    "range": [-5791.0, 110029], 
    "title": "RevPAR", 
    "type": "linear"
  }
}


fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'yearly_occupancy_rate.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)

In [ ]:
supply_demand_table['Year'] = supply_demand_table['Review Date'].dt.year
bedrooms_year = supply_demand_table[['cluster', 'Year', 'bedroom_count', 'property_count']].groupby(['cluster', 'Year']).max().reset_index()
bedrooms_year_total = bedrooms_year.groupby('Year').sum().reset_index()
bedrooms_year_total


trace_list = []
for cluster in cluster_nums:    
    trace = {
        "x": bedrooms_year[bedrooms_year['cluster'] == cluster]['Year'].astype('int'), 
        "y": bedrooms_year[bedrooms_year['cluster'] == cluster]['bedroom_count'],
        "mode": "lines", 
        "name": cluster,
        "text": cluster,
        }
    trace_list.append(trace)

trace = {
        "x": bedrooms_year_total['Year'], 
        "y": bedrooms_year_total['bedroom_count'],
        "mode": "lines", 
        "name": 'Total',
        "text": 'Total',
        }
trace_list.append(trace)

data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "Bedroom Supply", 
  "xaxis": {
    "autorange": True, 
    #"range": ["2014-08-31 15:42:21.1765", "2018-01-15 18:52:56.4706"], 
    "title": "Date", 
    #"type": "date"
  }, 
  "yaxis": {
    "autorange": True, 
#    "range": [-5791.0, 110029], 
    "title": "Bedroom count", 
    "type": "linear"
  }
}


fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'bedroom_count.html', show_link=False, auto_open=True)
plotly.offline.iplot(fig, show_link=False)


In [ ]:

regions_grouped['year'] = regions_grouped['Review Date'].dt.year

regions_grouped_yearly = regions_grouped[['cluster', 'year', 'projected_revenue']].groupby(['cluster', 'year']).sum().reset_index()

trace_list = []
for cluster in cluster_nums:    
    trace = {
        "x": regions_grouped_yearly[regions_grouped_yearly['cluster'] == cluster]['year'], 
        "y": regions_grouped_yearly[regions_grouped_yearly['cluster'] == cluster]['projected_revenue'], 
        "mode": "lines", 
        "name": cluster,
        "text": cluster,
        }
    trace_list.append(trace)

data = go.Data(trace_list)
layout = {
  "hovermode": "closest", 
  "title": "Revenue per cluster", 
  "xaxis": {
    "autorange": False, 
    "range": ["2014-08-31 15:42:21.1765", "2017-12-04 18:52:56.4706"], 
    "title": "Date", 
    "type": "date"
  }, 
  "yaxis": {
    "autorange": True, 
#    "range": [-5791.0, 110029], 
    "title": "Sum of Revenue", 
    "type": "linear"
  }
}


fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=viz_out+'yearly_revenue.html', show_link=False, auto_open=False)
plotly.offline.iplot(fig, show_link=False)

_______________________________________
## Draft Experiments Below
_______________________________________

In [ ]:

print(len(reviews))
print(len(tables_filtered['reviews']))
print(len(tables_filtered['reviews'].drop_duplicates()))
print(len(tables['reviews']))
print(len(tables['reviews'].drop_duplicates()))


In [ ]:
rev_df[rev_df['final_country_parse'] == 'British Indian Ocean Territory']#[['Country', 'City','final_country_parse']]

In [ ]:
rev_df = reviews_pre[[ 'Property ID', 'User ID', 'Review Date', 'Member Since', 'First Name', 
                   'Country', 'State', 'City', 'Description', 'School', 'Work', 
                   'country_after_parse', 'country_from_city_parse', 'final_country_parse' ]]



def correction(row):
    if row['Country'] == 'DE' or row['Country'] == 'GERMANY':
        return 'Germany'
    elif row['Country'] == 'SOUTH AFRICA':
        return 'South Africa'
    elif row['Country'] == 'FR':
        return 'France'
    elif row['Country'] == 'NETHERLANDS':
        return 'Netherlands'
    elif row['Country'] == 'UNITED STATES':
        return 'United States'
    elif row['Country'] == 'PT':
        return 'Portugal'
    elif row['Country'] == 'UNITED ARAB EMIRATES':
        return 'United Arab Emirates'
    elif row['Country'] == 'AU':
        return 'Australia'
    elif row['Country'] == 'GB':
        return 'United Kingdom'
    elif row['Country'] == 'BR':
        return 'Brazil'
    elif row['Country'] == 'INDIA':
        return 'India'
    else:
        return row['final_country_parse']
    


rev_df['final_country_parse'] = rev_df.apply(lambda x: correction(x), axis=1)
rev_df

In [ ]:
locations.columns

In [ ]:
yearly_revpar_overall['projected_revenue'] / yearly_revpar_overall['bedroom_count']

In [ ]:
#inertia_table.plot.line()
#plt.show()

In [ ]:
#for cluster in cluster_nums:    
#    print(regions_grouped[regions_grouped['cluster'] == cluster])
    
#regions_grouped['cluster'].dtype

cluster_nums[-1]

In [ ]:
#reviews_pre = pd.read_csv(preprocess_dir+'Portugal_Review_Standardized.csv', error_bad_lines=False)
len(reviews) # 1204312
len(reviews_pre) # 1204222
#tourists.sort_values('cluster')['cluster'].unique()

In [ ]:
len(tables_filtered['properties'])

In [ ]:
tables_filtered['properties'][['Annual Revenue LTM (USD)']].sum()

In [ ]:
len(reviews_lisbon)

In [ ]:
#properties['Bedrooms'].sum() # 209891.0

supply_demand_table.keys()
supply_demand_table['Year'] = supply_demand_table['Review Date'].dt.year
bedrooms_year = supply_demand_table[['cluster', 'Year', 'bedroom_count', 'property_count']].groupby(['cluster', 'Year']).max().reset_index()
bedrooms_year_total = bedrooms_year.groupby('Year').sum()
bedrooms_year_total
